<a id='top'></a>

# CSCI 3202, Spring 2018
# Assignment 5
# Due:  Wednesday 11 April 2018 by 12:00 PM

<br>

### Your name:

<br>

**Note:** Some packages to load, helper functions and unit tests are defined at [the bottom of this notebook](#helpers). They're also defined up here, because I care.

Shortcuts:  [top](#top) || [1](#p1) | [1a](#p1a) | [1b](#p1b) | [1c](#p1c) | [1d](#p1d) | [1e](#p1e) | [1f](#p1f) | [1g](#p1g) || [2](#p2) | [2a](#p2a) | [2b](#p2b) | [2c](#p2c) | [2d](#p2d) | [2e](#p2e) || [3](#p3) | [3a](#p3a) | [3b](#p3b) | [3c](#p3c) | [3d](#p3d) | [3e](#p3e) | [3f](#p3f) | [3g](#p3g) || [helpers](#helpers)

In [121]:
from scipy import stats
import unittest
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

---

<a id='p1'></a>[Back to top](#top)

## Problem 1:  Bayesian network to model heart disease

The following Bayesian network is based loosely on a study that examined heart disease risk factors in 167 elderly individuals in South Carolina.  This study is [linked here](https://piazza.com/class_profile/get_resource/jc4v74a5uu5wa/jeyiv7kvs7r7ck) and posted to Piazza under the Resources tab.  Note that this figure uses Y and N to represent Yes and No, whereas in class we used the equivalent T and F to represent True and False Boolean values.

<img src="http://www.cs.colorado.edu/~tonyewong/home/resources/hw05_bayesnet_heartdisease.png" style="width: 650px;"/>

<a id='p1a'></a>

### (1a) 

Create a `BayesNet` object to model this.  Below are the codes for the (conditional) probability `P` function and `BayesNode` class as well, that we used in class on Friday (16 March) to represent the variable nodes and calculate probabilities. You can code this however you want, subject to the following constraints:
1. the nodes are represented using the `BayesNode` class and can work with the `P` function for probabilities,
1. your `BayesNet` structure keeps track of which nodes are in the Bayes net, as well as
1. which nodes are the parents/children of which other nodes.

Some *suggested* skeleton codes for a class structure are given. The suggestions for methods to implement are in view of the fact that we will need to calculate some probabilities, which is going to require us to `find_node`s and `find_values` that nodes can take on.

In [122]:
## For the sake of brevity...
T, F = True, False

## From class:
def P(var, value, evidence={}):
    '''The probability distribution for P(var | evidence), 
    when all parent variables are known (in evidence)'''
    if len(var.parents)==1:
        # only one parent
        row = evidence[var.parents[0]]
    else:
        # multiple parents
        row = tuple(evidence[parent] for parent in var.parents)
    return var.cpt[row] if value else 1-var.cpt[row]

## Also from class:
class BayesNode:
    
    def __init__(self, name, parents, values, cpt):
        if isinstance(parents, str):
            parents = parents.split()
            
        if len(parents)==0:
            # if no parents, empty dict key for cpt
            cpt = {(): cpt}
        elif isinstance(cpt, dict):
            # if there is only one parent, only one tuple argument
            if cpt and isinstance(list(cpt.keys())[0], bool):
                cpt = {(v): p for v, p in cpt.items()}

        self.variable = name
        self.parents = parents
        self.cpt = cpt
        self.values = values
        self.children = []
        
    def __repr__(self):
        return repr((self.variable, ' '.join(self.parents)))    

    
##===============================================##
## Suggested skeleton codes for a BayesNet class ##
##===============================================##

class BayesNet:
    '''Bayesian network containing only boolean-variable nodes.'''

    def __init__(self, nodes):
        '''Initialize the Bayes net by adding each of the nodes,
        which should be a list BayesNode class objects ordered
        from parents to children (`top` to `bottom`, from causes
        to effects)'''
        
        # your code goes here...
        
        # Solution:
        self.nodes = []
        self.variables = []
        if nodes:
            for node in nodes:
                self.add(node)

                
    def add(self, node):
        '''Add a new BayesNode to the BayesNet. The parents should all
        already be in the net, and the variable itself should not be'''
        assert node.variable not in self.variables
        assert all((parent in self.variables) for parent in node.parents)
        
        # your code goes here...
        
        # Solution:
        self.nodes.append(node)
        self.variables.append(node.variable)
        for parent in node.parents:
            self.find_node(parent).children.append(node)

            
    def find_node(self, var):
        '''Find and return the BayesNode in the net with name `var`'''
        
        # your code goes here...
        
        # Solution:
        for n in self.nodes:
            if n.variable == var:
                return n
        raise Exception("No such variable: {}".format(var))

        
    def find_values(self, var):
        '''Return the set of possible values for variable `var`'''
        
        # your code goes here...
        
        # Solution:
        varnode = self.find_node(var)
        return varnode.values

    
    def __repr__(self):
        return 'BayesNet({})'.format(self.nodes)

In [123]:
# Solution:

# Define the nodes
Sm = BayesNode('Sm', '', [T,F], 0.2)
ME = BayesNode('ME', '', [T,F], 0.5)
HBP = BayesNode('HBP', ['Sm', 'ME'], [T,F], {(T, T): 0.6, (T, F): 0.72, (F, T): 0.33, (F, F): 0.51})
Ath = BayesNode('Ath', '', [T,F], 0.53)
FH = BayesNode('FH', '', [T,F], 0.15)
HD = BayesNode('HD', ['Ath', 'HBP', 'FH'], [T,F], 
               {(T, T, T): 0.92, (T, T, F): 0.91, (T, F, T): 0.81, (T, F, F): 0.77,
                (F, T, T): 0.75, (F, T, F): 0.69, (F, F, T): 0.38, (F, F, F): 0.23})
Ang = BayesNode('Ang', 'HD', [T,F], {T: 0.85, F: 0.4})
Rapid = BayesNode('Rapid', 'HD', [T,F], {T: 0.99, F: 0.3})

# Create a Bayes net with those nodes and connections
bnHeart = BayesNet([Sm, ME, HBP, Ath, FH, HD, Ang, Rapid])

#### Unit tests

In [124]:
tests_to_run = unittest.TestSuite()
tests_to_run.addTest(Tests_Problem1("test_onenode"))
tests_to_run.addTest(Tests_Problem1("test_twonode"))
unittest.TextTestRunner().run(tests_to_run)

..
----------------------------------------------------------------------
Ran 2 tests in 0.016s

OK


<unittest.runner.TextTestResult run=2 errors=0 failures=0>

<a/ id='p1b'></a>

### (1b)

Craft a function `get_prob(X, e, bn)` to return the **normalized** probability distribution of variable `X` in Bayes net `bn`, given the evidence `e`.  That is, return $P(X \mid e)$. The arguments are:
* `X` is some representation of the variable you are querying the probability distribution of. Either a string (the variable name from the `BayesNode` or a `BayesNode` object itself are good options.
* `e` is some representation of the evidence your probability is conditioned on. When given an empty argument (or `None`) for `e`, `get_prob` should return the marginal distribution $P(X)$.
* `bn` is your `BayesNet` object.

You may do this using the `enumeration` algorithm from class (pseudocode is in the book), or by brute force (i.e., use a few `for` loops). Either way, you should be using your `BayesNet` object to keep track of all the nodes and relationships between nodes so your `get_prob` function knows these things.

In [125]:
# Solution:

class PDF_discrete:
    '''Define a discrete probability distribution function.'''

    def __init__(self, varname='?', freqs=None):
        '''Create a dictionary of values - frequency pairs,
        then normalize the distribution to sum to 1.'''
        self.prob = {}
        self.varname = varname
        self.values = []
        if freqs:
            for (v, p) in freqs.items():
                self[v] = p
        self.normalize()

    def __getitem__(self, value):
        '''Given a value, return P[value]'''
        try:
            return self.prob[value]
        except KeyError:
            return 0

    def __setitem__(self, value, p):
        '''Set P[value] = p, input argument if '''
        if value not in self.values:
            self.values.append(value)
        self.prob[value] = p

    def normalize(self):
        '''Normalize the probability distribution and return it.
        If the sum of PDF values is 0, then return a 0'''
        total = sum(self.prob.values())
        if not isclose(total, 1.0):
            for value in self.prob:
                self.prob[value] /= total
        return self
    
def extend(s, var, val):
    """Copy the substitution s and extend it by setting var to val; return copy."""
    s2 = s.copy()
    s2[var] = val
    return s2

def get_prob(X, e, bn):
    '''Return the conditional probability distribution of variable X
    given evidence e, from BayesNet bn. [Figure 14.9]'''
    Q = PDF_discrete(X)
    for xi in bn.find_values(X):
        Q[xi] = enumerate_all(bn.variables, extend(e, X, xi), bn)
    return Q.normalize()


def enumerate_all(variables, e, bn):
    '''Return the sum of those entries in P(variables | e{others})
    consistent with e, where P is the joint distribution represented
    by bn, and e{others} means e restricted to bn's other variables
    (the ones other than variables). Parents must precede children in variables.'''
    if not variables:
        return 1.0
    Y, rest = variables[0], variables[1:]
    Ynode = bn.find_node(Y)
    if Y in e:
        # Y in evidence, so we know its value and just multiply
        return P(Ynode, e[Y], e) * enumerate_all(rest, e, bn)
    else:
        # Y not in evidence so we have to sum (Law of Total Prob.)    
        return sum(P(Ynode, y, e) * enumerate_all(rest, extend(e, Y, y), bn)
                   for y in bn.find_values(Y))

Use your `get_prob` function to calculate the following probabilities. Print them to the screen and compare to the original Bayes net figure given to make sure the output passes these "unit tests".

1. The marginal probability of `Family History` is $P(FH=T)=0.15$
2. The probability of *not* experiencing `Angina Pectoris`, given `Heart Disease` is observed, is $P(Ang=F \mid HD=T)=1-0.85=0.15$
3. The probability of `High Blood Pressure`, given a person does `Smoke and/or use Alcohol` but does not get `Moderate Exercise`, is $P(HBP=T \mid Sm=T, ME=F)=0.72$

In [126]:
# Solution:

p1 = get_prob(X='FH', e={}, bn=bnHeart)
print('P(FH)=',p1.prob)
print('')

p1 = get_prob(X='Ang', e={'HD' : T}, bn=bnHeart)
print('P(Ang | HD=T)=',p1.prob)
print('')

p1 = get_prob(X='HBP', e={'Sm' : T, 'ME' : F}, bn=bnHeart)
print('P(HBP | Sm=T, ME=T)=',p1.prob)

P(FH)= {True: 0.15, False: 0.85}

P(Ang | HD=T)= {True: 0.85, False: 0.15000000000000002}

P(HBP | Sm=T, ME=T)= {True: 0.7199999999999999, False: 0.28}


<a/ id='p1c'></a>

### (1c)

Calculate the probability of observing someone with `High Blood Pressure`, $P(HBP=T)$, *by hand*, showing all work in Markdown/LateX below.

**Solution:**

We use the Law of Total Probability to expand $P(HBP)$ in terms of the factors that affect it.  Namely, Smoking/Alcohol and Moderate Exercise, $Sm$ and $ME$.

$\begin{align*}
P(+hbp) &= \sum_{sm, me} P(+hbp \mid sm, me) P(sm, me) \\
        &= \sum_{sm} \sum_{me} P(+hbp \mid sm, me) P(sm) P(me) \\
        &= \sum_{sm} P(sm) \sum_{me} P(+hbp \mid sm, me) P(me) \\
        &= P(+sm)\left[P(+hbp \mid +sm, +me)P(+me) + P(+hbp \mid +sm, -me)P(-me)\right] + P(-sm)\left[P(+hbp \mid -sm, +me)P(+me) + P(+hbp \mid -sm, -me)P(-me)\right] \\
        &= 0.2\left[0.6\cdot 0.5 + 0.72\cdot (1-0.5)\right] + (1-0.2)\left[0.33\cdot 0.5 + 0.51\cdot (1-0.5)\right] \\
        &= \fbox{0.468}
\end{align*}$

**Verify** your calculation using your `get_prob` function.

In [127]:
# Solution:

p1 = get_prob(X='HBP', e={}, bn=bnHeart)
print('P(HBP)=',p1.prob)

P(HBP)= {True: 0.4680000000000001, False: 0.532}


<a/ id='p1d'></a>

### (1d)

Now calculate the following probabilities using your `get_prob` function.

[i] The probability of an arbitrary individual having `Heart Disease`, $P(HD=T)$

In [128]:
# Solution:

p1 = get_prob(X='HD', e={}, bn=bnHeart)
print('P(HD)={}'.format(p1.prob))

P(HD)={True: 0.6617765600000001, False: 0.33822343999999993}


[ii] The probability that an individual does *not* have `Heart Disease`, given that `Rapid Heartbeat` was observed, $P(HD=F \mid Rapid=T)$

In [129]:
# Solution:

p1 = get_prob(X='HD', e={'Rapid' : T}, bn=bnHeart)
print('P(HD | Rapid=T)={}'.format(p1.prob))

P(HD | Rapid=T)={True: 0.865895362727999, False: 0.13410463727200098}


[iii] The probability of an individual having `High Blood Pressure` if they have `Heart Disease` and a `Family History`, $P(HBP=T \mid HD=T, FH=T)$

In [130]:
# Solution:

p1 = get_prob(X='HBP', e={'HD' : T, 'FH' : T}, bn=bnHeart)
print('P(HBP | HD=T, FH=T)={}'.format(p1.prob))

P(HBP | HD=T, FH=T)={True: 0.5486791513343575, False: 0.4513208486656426}


[iv] The probability that an individual is a `Smoker/Alcohol User` if they have `Heart Disease`, $P(Sm=T \mid HD=T)$

In [131]:
# Solution:

p1 = get_prob(X='Sm', e={'HD' : T}, bn=bnHeart)
print('P(Sm | HD=T)={}'.format(p1.prob))

P(Sm | HD=T)={True: 0.2163440784303391, False: 0.7836559215696609}


[v] How would you expect the probability in [iv] to change if you also know the individual has `High Blood Pressure`?  Verify your hypothesis by calculating the relevant probability.

**Solution:** expect the probability that an individual is a smoker/alcohol user to go **up**, if we also know that this person has high blood pressure.  That's because smoking/alcohol use can **explain** the observation of high blood pressure.

In [132]:
# Solution:

p1 = get_prob(X='Sm', e={'HD' : T, 'HBP' : T}, bn=bnHeart)
print('P(Sm | HD=T, HBP=T)={}'.format(p1.prob))

P(Sm | HD=T, HBP=T)={True: 0.28205128205128205, False: 0.717948717948718}


[vi] How would you expect the probability in [v] to change if you also know that the individual does *not* get `Moderate Exercise` (in addition to having `Heart Disease` and `High Blood Pressure`)?  Explain your answer using concepts from class.  Verify your answer by calculating the relevant probability.

**Solution:** expect the probability that an individual is a smoker/alcohol user to go **down** because the fact that this individual does not get exercise can **explain away** the observation of high blood pressure, making the possible explanation of smoking/alcohol use relatively less likely.

In [133]:
# Solution:

p1 = get_prob(X='Sm', e={'HD' : T, 'HBP' : T, 'ME' : F}, bn=bnHeart)
print('P(Sm | HD=T, HBP=T, ME=F)={}'.format(p1.prob))

P(Sm | HD=T, HBP=T, ME=F)={True: 0.2608695652173913, False: 0.7391304347826088}


---

<a id='p2'></a>[Back to top](#top)

<img src="https://inhabitat.com/wp-content/blogs.dir/1/files/2014/02/norman-bike-riding-dog.png" style="width: 350px;"/>

## Problem 2:  Bayesian network to model decision-making

Let's consider using a Bayesian network to model our decision about whether or not to ride our bike to work today.  This decision depends heavily on the weather, so let's focus on that.

In class, we focused on Boolean variables.  For example, we might base our biking decision on whether or not it is raining.  But in reality, it probably matters *how hard* it is raining.  So suppose we break the variable `Rain` up into three discrete bins: `none`, `light` and `heavy`.

The temperature also factors into our decision.  There is definitely a sweet spot, where temperatures are neither too warm nor too cold, so it is very likely we would enjoy riding our bike.  So we can model the variable `Temperature` also using three discrete bins: `cold`, `moderate` and `warm`.

So a Bayesian network to model our decision for whether or not to bike to work could be as follows, where the first letter of each discrete bin is used to denote that variable value (i.e., `R=h` stands for heavy rain conditions).

<img src="http://www.cs.colorado.edu/~tonyewong/home/resources/bayesnet_biking2.png" style="width: 650px;"/>

<a/ id='p2a'></a>

### (2a)

Modify the `P` probability function to be able to handle these ternary parent nodes.

In [134]:
# modified function for conditional probabilities,
# to handle ternary (or more) case
def P(var, value, evidence={}):
    '''The probability distribution for P(var | evidence), 
    when all parent variables are known (in evidence)'''
    if len(var.parents)==1:
        # only one parent
        row = evidence[var.parents[0]]
    else:
        # multiple parents
        row = tuple(evidence[parent] for parent in var.parents)
    if len(var.parents)==0:
        return var.cpt[row][value] if value in var.cpt[row].keys() else 1-sum(var.cpt[row].values())
    return var.cpt[row] if value else 1-var.cpt[row]

Set up `BayesNode` objects for each of `Rain`, `Temp` and `Bike`, and create a `BayesNet` object to model the Bayesian network for this decision.  Again, you can use whatever structure you wish for your `BayesNet`, but please use the `BayesNode` class.  You may need to make minor modifications to the `BayesNode` class (e.g., changing/adding attributes), although none are strictly necessary.

In [135]:
# Set up the Bayes net
n,l,h,c,m,w,T,F = 'None','Light','Heavy','Cold','Moderate','Warm',True,False

rain = BayesNode('Rain', '', [n,l,h], {n : 0.8, l : 0.15})
temp = BayesNode('Temp', '', [c,m,w], {c : 0.3, m : 0.6 })
bike = BayesNode('Bike', ['Rain', 'Temp'], [T, F], {(n,c) : 0.7, (n,m) : 0.99, (n,w) : 0.9,
                                                    (l,c) : 0.4, (l,m) : 0.6 , (l,w) : 0.5,
                                                    (h,c) : 0.2, (h,m) : 0.4 , (h,w) : 0.3})
bnRide = BayesNet([rain, temp, bike])

**Verify** that your modified probability function `P` is working by checking the following "unit tests". Print the output to screen and compare to what you expect from the figure above.

1. The marginal probability of no rain is $P(Rain=n)=0.8$
1. The marginal probability of light rain is $P(Rain=l)=0.15$
1. The marginal probability of heavy rain is $P(Rain=h)=0.05$
1. The probability of biking given that it is raining heavily and the temperature is cold, is $P(Bike=T \mid Rain=h, Temp=c)=0.2$

In [136]:
## Solution:

print('P(R=n)={}'.format(P(rain, n)))
print('P(R=l)={}'.format(P(rain, l)))
print('P(R=h)={}'.format(P(rain, h)))
print('P(B=T | R=h, T=c)={}'.format(P(bike, T, {'Rain' : h, 'Temp' : c})))

P(R=n)=0.8
P(R=l)=0.15
P(R=h)=0.04999999999999993
P(B=T | R=h, T=c)=0.2


<a/ id='p2b'></a>

### (2b)

Make any necessary modifications to your `get_prob` function from Problem 1, so that you can use it to calculate marginal probabilities and conditional probabilities for this problem. It is possible that your function does not require any modifications.

Use `get_prob` to calculate $P(Bike)$, the probability distribution for whether or not you will ride your bike on any given day.

In [140]:
# Solution:

p1 = get_prob(X='Bike', e={}, bn=bnRide)
print(p1.prob)

{True: 0.8112, False: 0.18880000000000002}


Use `get_prob` to calculate the probability that you will ride your bike, given that it is lightly raining.

In [141]:
# Solution:

p1 = get_prob(X='Bike', e={'Rain' : l}, bn=bnRide)
print(p1.prob)

{True: 0.5299999999999999, False: 0.47}


<a/ id='p2c'></a>

### (2c)

We are trapped indoors because some jerk gave us a ton of Intro to Artificial Intelligence homework to do.  Suppose we look out the window and see people biking. They sure do look like they're having fun! *Given* this information, we can actually make inferences regarding the temperature outside!  What is the probability distribution for temperature, given that we observe people biking?

First, compute this using your `get_prob` function.

In [145]:
# Solution:

p1 = get_prob(X='Temp', e={'Bike' : T}, bn=bnRide)
print(p1.prob)

{'Cold': 0.23298816568047337, 'Moderate': 0.6671597633136095, 'Warm': 0.09985207100591724}


<a/ id='p2d'></a>

### (2d)

Confirm your answer to **2c** by hand, showing *all* relevant work below in a LateX/Markdown cell.

**Solution:**

Hidden variable(s) is just Rain, so we need a sum over all the possible values of Rain. $\alpha$ is a normalization constant that we'll figure out in order to make $\sum_t P(t \mid +b) = 1$, like a good wholesome probability distribution (since *one* of the possible values of Temperature, $t$, must be the case).

$\begin{align*}
P(t \mid +b) &= \alpha \sum_r \prod_i P(x_i \mid \text{parents}(X_i)) \\
             &= \alpha \sum_r P(r)P(t)P(+b \mid r,t) \\
             &= \alpha P(t) \left[P(r=n) P(+b \mid r=n, t) + P(r=l) P(+b \mid r=l, t) + P(r=h) P(+b \mid r=h, t)\right] \\
\end{align*}$

This equation holds for each of $P(t=c \mid +b)$, $P(t=m \mid +b)$ and $P(t=h \mid +b)$.  **The plan:** calculate each of these, then figure out $\alpha$ so the three of these conditional probabilities sum to 1, then calculate (using $\alpha$ exactly what each is.

$\begin{align*}
P(t=c \mid +b) &= \alpha P(t=c) \left[P(r=n) P(+b \mid r=n, t=c) + P(r=l) P(+b \mid r=l, t=c) + P(r=h) P(+b \mid r=h, t=c)\right] \\
               &= \alpha \cdot 0.3 [0.8\cdot 0.7 + 0.15\cdot 0.4 + 0.05\cdot 0.2] \\
               &= 0.189\alpha
\end{align*}$

$\begin{align*}
P(t=m \mid +b) &= \alpha P(t=m) \left[P(r=n) P(+b \mid r=n, t=m) + P(r=l) P(+b \mid r=l, t=m) + P(r=h) P(+b \mid r=h, t=m)\right] \\
               &= \alpha \cdot 0.6 [0.8\cdot 0.99 + 0.15\cdot 0.6 + 0.05\cdot 0.4] \\
               &= 0.5412\alpha
\end{align*}$

$\begin{align*}
P(t=w \mid +b) &= \alpha P(t=w) \left[P(r=n) P(+b \mid r=n, t=w) + P(r=l) P(+b \mid r=l, t=w) + P(r=h) P(+b \mid r=h, t=w)\right] \\
               &= \alpha \cdot 0.1 [0.8\cdot 0.9 + 0.15\cdot 0.5 + 0.05\cdot 0.3] \\
               &= 0.081\alpha
\end{align*}$

Now we calculate the normalization constant, $\alpha$:

$\begin{align*}
1 &\stackrel{\heartsuit}{=} P(t=c \mid +b) + P(t=m \mid +b) + P(t=w \mid +b) \\
  &= 0.189\alpha + 0.5412\alpha + 0.081\alpha \\
\end{align*}$

So $\alpha = 1.23274$, and we get:

$$P(t=c \mid +b) = 0.233$$
$$P(t=m \mid +b) = 0.667$$
$$P(t=w \mid +b) = 0.100$$

which is consistent with **2c**.  Horray!

<a/ id='p2e'></a>

### (2e)

Finally, confirm your confirmation of the probability distribution for `Temp` by using approximate Bayesian computation and 10,000 samples.  That is, use the **prior sampling** and **"rejection sampling"** techniques from class to estimate the probabilities associated with each possible value for `Temp`, given that there are people biking outside.

In [173]:
# Solution:

n_sample = 10000

# sample from the priors
dfSample = pd.DataFrame({'Rain' : np.random.choice([n,l,h], size=n_sample, p=[0.8, 0.15, 0.05]),
                         'Temp' : np.random.choice([c,m,w], size=n_sample, p=[0.3, 0.6, 0.1])})

# get that handy-dandy sample_2parent function from our in-class notebook
def sample_2parent(row, variable, parents):
    return np.random.choice([T,F], p=[P(variable, T, {parents[0]: row[parents[0]], parents[1]: row[parents[1]]}), 
                                      P(variable, F, {parents[0]: row[parents[0]], parents[1] : row[parents[1]]})])

# sample from distribution of bike, given the parents
dfSample['Bike'] = dfSample.apply(sample_2parent, axis=1, variable=bike, parents=bike.parents)

# estimate of P(t=c | +b) is [# rows with t=c and +b]/[# rows with +b]
p_cold = dfSample.loc[(dfSample['Temp']==c) & (dfSample['Bike']==T)].count()[0]/dfSample.loc[(dfSample['Bike']==T)].count()[0]

# similarly, estimate of P(t=m | +b) is [# rows with t=m and +b]/[# rows with +b]
p_mod = dfSample.loc[(dfSample['Temp']==m) & (dfSample['Bike']==T)].count()[0]/dfSample.loc[(dfSample['Bike']==T)].count()[0]

# and estimate of P(t=w | +b) is [# rows with t=w and +b]/[# rows with +b]
p_warm = dfSample.loc[(dfSample['Temp']==w) & (dfSample['Bike']==T)].count()[0]/dfSample.loc[(dfSample['Bike']==T)].count()[0]

print('P(T=c | +b) = {:0.4f} // P(T=m | +b) = {:0.4f} // P(T=w | +b) = {:0.4f}'.format(p_cold, p_mod, p_warm))

P(T=c | +b) = 0.2375 // P(T=m | +b) = 0.6597 // P(T=w | +b) = 0.1028


As a "Unit Test", check what the probability of riding your bike is, given no other information.  Make sure this approximately matches your answer to **2b**.

In [174]:
# Solution:

p_bike = dfSample["Bike"].sum()/n_sample
print('Estimate P(Bike) = {:0.4f}'.format(p_bike))

Estimate P(Bike) = 0.8090


---

<a id='p3'></a>

## Problem 3:  Markov models - random walk to Taco Bell

Your friend Chris went to a party last night and drank way too much Fun Juice.  Chris is feeling a bit hungry, so he leaves the party, which is at the corner of 6th Street East and 3rd Street North, and heads for Taco Bell, which is located at the corner of 2nd Street East and 1st Street North.  A figure depicting this neighborhood is given below.

<img src="http://www.cs.colorado.edu/~tonyewong/home/resources/random_walk_to_taco_bell.png" style="width: 650px;"/>

Fun Juice makes Chris' sense of direction quite poor, so at each intersection along his way, he picks any one of the available directions with equal probability.  Chris at least knows not to go north of 4th Street North, south of 0th Street North, east of 7th Street East, or west of 0th Street East, and has the common decency not to cut through anyone's yard (i.e., he only walks along streets).

Suppose Chris only cares about traveling between from one intersection to another, and considers one *move* to be walking one block, from one intersection to an adjacent intersection.

Since this grid is precisely a Cartesian coordinate grid, let the bottom-left corner of the neighborhood be represented by $(0,0)$.  Then the available moves from that location are to walk East (right, in the $+x$ direction) to $(1,0)$ or North to $(0,1)$.

<a id='p3a'></a>

### (3a)

Create a class for `Neighborhood` and for `Agent`, to represent the neighborhood and the agent trying to get to Taco Bell:

`Neighborhood(n_northsouth, n_eastwest, taco_bell)`:
* `n_northsouth` and `n_eastwest` provide the number of streets running north-south and the number of streets running east-west, respectively.  In the given figure, there are 5 streets running east-west, for example.
* `taco_bell` is a tuple providing the coordinates of Taco Bell in this neighborhood.
* has attributes for:
  * number of streets running north-south
  * number of streets running east-west
  * all of the intersections in the neighborhood
  * the location of the Taco Bell in the neighborhood
* Implement in your code a check to make sure the location of the Taco Bell is within the neighborhood's coordinates.  Assume that the south-west corner of the neighborhood is always $(0,0)$.

`Agent(name, loc, neighborhood)`:
* In the constructor, provide the agent with a `name` (string), initial location as a tuple (`loc`), and a `Neighborhood` to live in. Store these as attributes.
* Fill in the rest of the needed methods for the agent:
  * `available_moves()` returns a list of tuples, representing the locations the agent can walk to from its current location
  * `random_move()` returns one of the possible moves
  * `walk(move)` updates the agent's location, if they make the given argument `move`
  * `at_taco_bell()` returns True if the agent is at the same intersection as the neighborhood Taco Bell, and False otherwise

In [425]:
class Neighborhood:
    def __init__(self, n_northsouth, n_eastwest, taco_bell):
        '''Set up the layout of the neighborhood by giving the # streets
        running North/South (n_northsouth) and the # streets running 
        East/West). Based on these, store all the available locations 
        (intersections) in the neighborhood, and make sure the given
        coordinates for Taco Bell are valid'''
        
        # your code goes here...
        
        # Solution:
        self.n_ns = n_northsouth
        self.n_ew = n_eastwest
        self.locations = [(col, row) for col in range(n_northsouth) for row in range(n_eastwest)]
        self.tb = taco_bell
        assert taco_bell in self.locations, 'Taco Bell must be within the neighborhood domain'
        
class Agent:
    def __init__(self, name, loc, neighborhood):
        
        # your code goes here...
        
        # Solution:
        self.name = name
        self.loc = loc
        self.nh = neighborhood
        
    def available_moves(self):
        '''Return a list of available intersections the agent can move to,
        based on the layout of the agent's neighborhood'''
        
        # your code goes here...
        
        # Solution:
        moves = []
        if self.loc[0] > 0:
            # can move West
            moves.append((self.loc[0]-1, self.loc[1]))
        if self.loc[0] < self.nh.n_ns-1:
            # can move East
            moves.append((self.loc[0]+1, self.loc[1]))
        if self.loc[1] > 0:
            # can move South
            moves.append((self.loc[0], self.loc[1]-1))
        if self.loc[1] < self.nh.n_ew-1:
            # can move North
            moves.append((self.loc[0], self.loc[1]+1))
        return moves

    def random_move(self):
        '''Return a random move out of the available moves
        from the agent`s current location'''
        
        # your code goes here...
        
        # Solution:
        moves = self.available_moves()
        n_moves = len(self.available_moves())
        return moves[np.random.randint(n_moves)]
    
    def walk(self, move):
        '''Update the agent to a new location'''
        
        # your code goes here...
        
        # Solution:
        self.loc = move
        
    def at_taco_bell(self):
        '''Return True if the agent is at Taco Bell, and False otherwise'''
        
        # your code goes here...
        
        # Solution:
        return self.loc==self.nh.tb
    
    def __repr__(self):
        return '{} at {}'.format(self.name, self.loc)

#### Unit tests

In [426]:
tests_to_run = unittest.TestSuite()
tests_to_run.addTest(Tests_Problem3('test_moves_corner'))
tests_to_run.addTest(Tests_Problem3('test_moves_center'))
tests_to_run.addTest(Tests_Problem3('test_tacos'))
unittest.TextTestRunner().run(tests_to_run)

...
----------------------------------------------------------------------
Ran 3 tests in 0.005s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>

<a/ id='p3b'></a>

### (3b)

Create a neighborhood and Chris agent to represent the situation above.

Then, run an ensemble of 1,000 simulations to obtain a sample for the number of blocks Chris must travel in order to arrive at Taco Bell at (2,1), starting from the party at (6,3).  Report the expected number of blocks traveled.

In [427]:
# Solution:

taco_bell = (2,1)
nh = Neighborhood(n_eastwest=5, n_northsouth=8, taco_bell=taco_bell)
n_sample = 1000
sample_blocks = []
for n in range(n_sample):
    chris = Agent('Chris', (6,3), nh)
    arrived = False
    blocks = 0
    while not arrived:
        chris.walk(chris.random_move())
        blocks += 1
        arrived = chris.at_taco_bell()
    sample_blocks.append(blocks)
    
print(np.mean(sample_blocks))

81.622


**Reflection:**  The sequence of states (coordinates) that Chris passed through in his travels is a **Markov chain** - each new state depended only on the previous one.  This process, called a **random walk**, can be a useful way to explore a state space.


<a/ id='p3c'></a>

### (3c)

Let us explore one of the characteristics of a Markov chain that we often find ourselves interested in:  the **expected time of first return** to a state.  In particular, let's examine Chris' **expected time of first return to Taco Bell**.

Build an ensemble of 1,000 simulations of how many blocks Chris must travel to return to Taco Bell, given that he starts at Taco Bell in the neighborhood depicted in the figure above.

We can estimate the expected time of first return to Taco Bell using the **mean** number of blocks Chris must travel in order to make it back to those tasty tacos. Report this estimate based on your simulation results.

In [428]:
# Solution:

taco_bell = (2,1)
nh = Neighborhood(n_eastwest=5, n_northsouth=8, taco_bell=taco_bell)
n_sample = 1000
sample_blocks = []
for n in range(n_sample):
    chris = Agent('Chris', taco_bell, nh)
    arrived = False
    blocks = 0
    while not arrived:
        chris.walk(chris.random_move())
        blocks += 1
        arrived = chris.at_taco_bell()
    sample_blocks.append(blocks)

print(np.mean(sample_blocks))

31.638


<a id='p3d'></a>

### (3d)

Your friend Dan also is leaving the party.  Dan leaves just after Chris.  Dan, however, has a preference for heading west (left, in the $-x$ direction) when he can, for that is where Ralphie roams.  In fact, his preference for being nearer to Ralphie is so strong that if it is possible for him to move west, then he does so 60% of the time, with the other 40% of the probability distributed equally among his other options.

Revise the `Agent` class - or better yet, *sub-class it*, to represent Dan's strange yet endearing decision-making for which random direction to move in.

In [429]:
# Solution:

class AgentDan(Agent):
    
    def sortof_random_move(self):
        
        moves = self.available_moves()
        n_moves = len(self.available_moves())
        
        # check if a west-ward move is in there
        west = [move[0]-self.loc[0]==-1 for move in moves]
        if not any(west):
            return moves[np.random.randint(n_moves)]            
        else:
            p_west = 0.6
            p_notwest = (1-p_west)/(n_moves-1)
            probs = []
            for i in range(n_moves):
                if west[i]:
                    probs.append(p_west)
                else:
                    probs.append(p_notwest)
            return moves[np.random.choice(list(range(n_moves)), p=probs)]

Now simulate 1,000 samples for how many blocks Dan will need to travel in order to make it to Taco Bell from the party.  Report the expected number of blocks Dan must travel to get some delicious chalupas.

In [430]:
# Solution:

taco_bell = (2,1)
nh = Neighborhood(n_eastwest=5, n_northsouth=8, taco_bell=taco_bell)
n_sample = 1000
sample_blocks = []
for n in range(n_sample):
    dan = AgentDan('Dan', (6,3), nh)
    arrived = False
    blocks = 0
    while not arrived:
        dan.walk(dan.sortof_random_move())
        blocks += 1
        arrived = dan.at_taco_bell()
    sample_blocks.append(blocks)
    
print(np.mean(sample_blocks))

74.764


<a/ id='p3e'></a>

### (3e)

What is Dan's expected time of first return to Taco Bell, as measured by the number of blocks traveled?

In [431]:
# Solution:

taco_bell = (2,1)
nh = Neighborhood(n_eastwest=5, n_northsouth=8, taco_bell=taco_bell)
n_sample = 1000
sample_blocks = []
for n in range(n_sample):
    dan = AgentDan('Dan', taco_bell, nh)
    arrived = False
    blocks = 0
    while not arrived:
        dan.walk(dan.sortof_random_move())
        blocks += 1
        arrived = dan.at_taco_bell()
    sample_blocks.append(blocks)

print(np.mean(sample_blocks))

53.502


### (3f)

Consider the smaller neighborhood depicted below, with only 3 north-south streets, 2 east-west streets, and a Taco Bell located at $(1,1)$.

<img src="http://www.cs.colorado.edu/~tonyewong/home/resources/random_walk_to_taco_bell_small.png" style="width: 250px;"/>

There are 6 distinct states in the state space, one for each of the 6 intersections.  Recall that the transition probability matrix denotes the probability of moving from the state given by the row of the matrix to the state given by the column.

Recall that the transition probability matrix is given by

$$T = \left[\begin{array}{cccccc} 
  q((0,0),(0,0)) & q((0,0),(1,0)) & q((0,0),(2,0)) & q((0,0),(0,1)) & q((0,0),(1,1)) & q((0,0),(2,1)) \\
  q((1,0),(0,0)) & q((1,0),(1,0)) & q((1,0),(2,0)) & q((1,0),(0,1)) & q((1,0),(1,1)) & q((1,0),(2,1)) \\
  q((2,0),(0,0)) & q((2,0),(1,0)) & q((2,0),(2,0)) & q((2,0),(0,1)) & q((2,0),(1,1)) & q((2,0),(2,1)) \\
  q((0,1),(0,0)) & q((0,1),(1,0)) & q((0,1),(2,0)) & q((0,1),(0,1)) & q((0,1),(1,1)) & q((0,1),(2,1)) \\
  q((1,1),(0,0)) & q((1,1),(1,0)) & q((1,1),(2,0)) & q((1,1),(0,1)) & q((1,1),(1,1)) & q((1,1),(2,1)) \\
  q((2,1),(0,0)) & q((2,1),(1,0)) & q((2,1),(2,0)) & q((2,1),(0,1)) & q((2,1),(1,1)) & q((2,1),(2,1)) \\  
\end{array} \right]$$

where $q(s_1, s_2)$ is the probability of moving from state $s_1$ to state $s_2$ in one step.

**For example:**  If Dan is currently at $(1,0)$, then:
* $q((1,0),(0,0))=0.6$
* $q((1,0),(2,0))=0.2$
* $q((1,0),(1,1))=0.2$
* $q((1,0),(0,1))=q((1,0),(2,1))=0$
* $q((1,0),(1,0))=0$ because Dan must move somewhere.

This row is already filled in for you.

Finish filling in the rest of the transition probability matrices below for each of Chris and Dan.

In [ ]:
T_dan = np.array([[   ,    ,    ,    ,    ,    ],
                  [0.6, 0  , 0.2, 0  , 0.2, 0  ],
                  [   ,    ,    ,    ,    ,    ],
                  [   ,    ,    ,    ,    ,    ],
                  [   ,    ,    ,    ,    ,    ],
                  [   ,    ,    ,    ,    ,    ]])

T_chris = np.array([[   ,    ,    ,    ,    ,    ],
                    [   ,    ,    ,    ,    ,    ],
                    [   ,    ,    ,    ,    ,    ],
                    [   ,    ,    ,    ,    ,    ],
                    [   ,    ,    ,    ,    ,    ],
                    [   ,    ,    ,    ,    ,    ]])

In [388]:
# Solution:

T_dan = np.array([[0  , 0.5, 0  , 0.5, 0  , 0  ],
                  [0.6, 0  , 0.2, 0  , 0.2, 0  ],
                  [0  , 0.6, 0  , 0  , 0  , 0.4],
                  [0.5, 0  , 0  , 0  , 0.5, 0  ],
                  [0  , 0.2, 0  , 0.6, 0  , 0.2],
                  [0  , 0  , 0.4, 0  , 0.6, 0  ]])


T_chris = np.array([[0  , 0.5, 0  , 0.5, 0  , 0  ],
                    [1/3, 0  , 1/3, 0  , 1/3, 0  ],
                    [0  , 0.5, 0  , 0  , 0  , 0.5],
                    [0.5, 0  , 0  , 0  , 0.5, 0  ],
                    [0  , 1/3, 0  , 1/3, 0  , 1/3],
                    [0  , 0  , 0.5, 0  , 0.5, 0  ]])

Use your transition probability matrices to estimate the probabilities (independently) that each of Chris and Dan will be at Taco Bell in 2 moves, given that they starts at $(0,0)$.

In [411]:
# Solution:

print('Dan probability = {:0.4f}'.format(np.linalg.matrix_power(T_dan, 2)[0][4]))
print('Chris probability = {:0.4f}'.format(np.linalg.matrix_power(T_chris, 2)[0][4]))

Dan probability = 0.3500
Chris probability = 0.4167


What is the probability that either are at Taco Bell after 3 moves, if they start from $(0,0)$?  Why is this?

**Solution:**

Say anything about parity, and/or that Chris and Dan must keep moving and can only be at Taco Bell after an **even** number of moves.

<a/ id='p3g'></a>

### (3g)

If Chris and Dan both spend all of their time walking around and eating at Taco Bell, what is the long-run proportion of Chris and Dan's time that each will spend at Taco Bell?  That is, if you run the random walk simulation for a very, very long time, what is your estimate for the proportion of states in the Markov chain that are at Taco Bell?

First, estimate these probabilities using long Markov chain random walk simulations for each of Chris and Dan. About 100,000 moves should be enough to stabilize your estimates.

In [433]:
# Solution:

taco_bell = (1,1)
nh = Neighborhood(n_eastwest=2, n_northsouth=3, taco_bell=taco_bell)
n_step = 100000
tacos = 0
dan = AgentDan('Dan', (0,0), nh)
for n in range(n_step):
    dan.walk(dan.sortof_random_move())
    if dan.at_taco_bell():
        tacos += 1

print('Dan probability of being at Taco Bell = {:0.4f}'.format(tacos/n_step))


tacos = 0
chris = Agent('Chris', (0,0), nh)
for n in range(n_step):
    chris.walk(chris.random_move())
    if chris.at_taco_bell():
        tacos += 1

print('Chris probability of being at Taco Bell = {:0.4f}'.format(tacos/n_step))

Dan probability of being at Taco Bell = 0.1987
Chris probability of being at Taco Bell = 0.2150


Now estimate these probabilities using your transition probability matrices.  You should notice something a little bit strange when you compare these results against those from the simulations above.  Provide a couple sentences interpreting these results.

In [424]:
# Solution:

print('Dan transitions after 1000 steps:')
print(np.linalg.matrix_power(T_dan, 1000))
print('')
print('Dan transitions after 1001 steps:')
print(np.linalg.matrix_power(T_dan, 1001))

Dan transitions after 1000 steps:
[[ 0.47368421  0.          0.13157895  0.          0.39473684  0.        ]
 [ 0.          0.39473684  0.          0.47368421  0.          0.13157895]
 [ 0.47368421  0.          0.13157895  0.          0.39473684  0.        ]
 [ 0.          0.39473684  0.          0.47368421  0.          0.13157895]
 [ 0.47368421  0.          0.13157895  0.          0.39473684  0.        ]
 [ 0.          0.39473684  0.          0.47368421  0.          0.13157895]]

Dan transitions after 1001 steps:
[[ 0.          0.39473684  0.          0.47368421  0.          0.13157895]
 [ 0.47368421  0.          0.13157895  0.          0.39473684  0.        ]
 [ 0.          0.39473684  0.          0.47368421  0.          0.13157895]
 [ 0.47368421  0.          0.13157895  0.          0.39473684  0.        ]
 [ 0.          0.39473684  0.          0.47368421  0.          0.13157895]
 [ 0.47368421  0.          0.13157895  0.          0.39473684  0.        ]]


**Solution:** Dan and Chris can only get to Taco Bell with an ***even*** number of moves.  Thus, half of the transition probabilities from somewhere to Taco Bell are 0, and the result from the simulations is the ***mean*** of the column of the transition probability matrices (raised to a power large enough to have reached the stationary distribution), which is the 5th column.

I'm happy if you say anything consistent with the proper interpretation of the $n^{th}$ power of a transition probability matrix.  That is, **IF** Dan starts in a state "with the same parity as Taco Bell" (one where Taco Bell is accessible in $n$ moves), then probability he is at Taco Bell after $n$ moves is about 39%. Since you do not have a prior distribution for the initial states for Chris and Dan, you cannot say for sure what the exact long-run probability is. But if all states were equally likely, then it would be these entires/2, or **exactly what we got above from sampling!**

Really, we should be using the law of total probability to estimate $P(TB)$, conditioned on our starting state, $s_0$:

$\begin{align*} P(TB) &= P(TB \mid s_0=(0,0)) P(s_0 = (0,0)) + P(TB \mid s_0=(0,1)) P(s_0 = (0,1)) + P(TB \mid s_0=(1,0)) P(s_0 = (1,0)) + P(TB \mid s_0=(1,1)) P(s_0 = (1,1)) + P(TB \mid s_0=(2,0)) P(s_0 = (2,0)) + P(TB \mid s_0=(2,1)) P(s_0 = (2,1)) \\
\end{align*}$

<br><br><br>

<a id='helpers'></a>

---

[Back to top](#top)

## Some things that might be useful

Easiest way to start:  Click this cell, go to "Cell" in the toolbar above, and click "Run All Below"

In [49]:
from scipy import stats
import unittest
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Unit tests

In [394]:
class Tests_Problem1(unittest.TestCase):
    def setUp(self):
        self.p1 = BayesNode('p1', '', [T,F], 0.3)
        self.p2 = BayesNode('p2', '', [T,F], 0.6)
        self.c  = BayesNode('c', ['p1', 'p2'], [T,F], {(T,T):0.1, (T,F):0.2, (F,T):0.3, (F,F):0.4})
    def test_onenode(self):
        self.assertEqual(P(self.p1, T), 0.3)
    def test_twonode(self):
        self.assertEqual(P(self.c, F, {'p1':T, 'p2':F}), 0.8)
        
class Tests_Problem3(unittest.TestCase):
    def test_moves_corner(self):
        nh = Neighborhood(2,2, (1,1))
        chris = Agent('Chris', (0,0), nh)
        self.assertTrue(((1,0) in chris.available_moves()) and ((0,1) in chris.available_moves()))
    def test_moves_center(self):
        nh = Neighborhood(3,3, (1,1))
        chris = Agent('Chris', (1,1), nh)
        self.assertTrue(((1,0) in chris.available_moves()) and ((0,1) in chris.available_moves()) and
                        ((1,2) in chris.available_moves()) and ((2,1) in chris.available_moves()))
    def test_tacos(self):
        nh = Neighborhood(3,3, (1,1))
        chris = Agent('Chris', (1,1), nh)
        self.assertTrue(chris.at_taco_bell())

[Back to top](#top)